In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
import psutil 
from PIL import Image
import numpy as np 

In [2]:
model = load_model("./data/model/face_model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 46, 46, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 44, 44, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 44, 44, 64)        0

In [3]:
def split_model(model, select_layer):
    split_layer = model.get_layer(select_layer)
    model1 = Model(inputs=model.input, outputs=split_layer.output)

    model2_input = split_layer.output
    model2_layers = model.layers[model.layers.index(split_layer) + 1:]
    x = model2_input
    for layer in model2_layers:
        x = layer(x)

    model2 = Model(inputs=model2_input, outputs=x)

    return model1, model2

In [4]:
model1, model2 = split_model(model, "conv2d_1")

In [5]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 46, 46, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
Total params: 37,824
Trainable params: 37,696
Non-trainable params: 128
_______________________________________________________

In [6]:
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 44, 44, 64)]      0         
                                                                 
 batch_normalization_1 (Batc  (None, 44, 44, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 44, 44, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 32)        1846

In [10]:
def dynamic_allocate(model):
    layers = ["conv2d_1", "conv2d_2", "conv2d_3"]
    cpu_usage = psutil.cpu_percent()
    if cpu_usage == 0:
        select_layer = layers[2]
    elif cpu_usage < 10 and cpu_usage > 0: 
        select_layer = layers[1]
    else: 
        select_layer = layers[0]

    model1, model2 = split_model(model, select_layer)
    return model1, model2

In [11]:
def run_model(photo): 
    model = load_model("./data/model/face_model.h5")
    model1, model2 = dynamic_allocate(model)
    
    img = Image.open(photo).convert("L").resize((48, 48))
    img_array = np.array(img).reshape(1, 48, 48, 1).astype('float32')
    result = model1.predict(img_array)
    return result

In [12]:
result = run_model("./data/test_photo/photo_test.jpg")
print(result.shape)

1/1 [==============================] - 0s 67ms/step
(1, 20, 20, 32)


2024-11-29 13:07:10.030098: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
